In [1]:
import subprocess
import json 
from langchain_ollama.llms import OllamaLLM
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from pydantic import BaseModel, Field,field_validator
from langchain_ollama import ChatOllama


In [2]:
def ejecutar_puppeteer(usuario, contrasena):
    print(usuario,contrasena)
    try:
        # Ejecuta el script Node.js y pasa usuario y contraseña como argumentos
        subprocess.run(
            ["/home/bsrosasm/.nvm/versions/node/v20.17.0/bin/node", "iniciosesion.js", usuario, contrasena], 
            check=True
        )
        return "pupeteer conectado de manera eficaz"
    except subprocess.CalledProcessError as e:
        return f"Error al ejecutar Puppeteer: {str(e)}"

In [3]:
ll3=ChatOllama(model="llama3.1",temperature=1.5)


In [4]:
class Usuario(BaseModel):
    usuario:str=Field(description="usuario",required=True)
    contrasena:str=Field(description="contrasena, si es un numero se trendra que hacer la conversion a string",required=True)
    @field_validator("contrasena", mode="before")
    def convertir_contrasena_a_string(cls, value):
        return str(value)

In [5]:
custom_prompt_template = """
Actúa como un asistente que ayuda a los usuarios con inicios de sesión.
El usuario dice: "{accion}"

Extrae el nombre de usuario y la contraseña que menciona el usuario.
Si la información está disponible en la frase, devuélvela de la siguiente forma:
- Usuario: [nombre de usuario]
- Contraseña: [contraseña]

Si no está claro, pidele que la ingrese, si no la ingresa, di que hubo un error y que no pudiste
obtener el usuario ni contraseña"
"""


In [6]:
prompt=PromptTemplate(template=custom_prompt_template, input_variables=["accion"])

In [7]:
structured_llm = ll3.with_structured_output(Usuario)
chain = prompt | structured_llm


In [243]:
#action = "quiero iniciar sesion, pero no se como,mi contraseña es 123tasd y mi usuario es admin@gmail.com y "

#respuesta = chain.invoke({"accion": action})
#print(respuesta.usuario)

In [9]:
def main():
    # Se obtiene la acción inicial
    while True:
        accion = input("Dime qué quieres hacer (escribe 'salir' para terminar): ")

        # Salir si el usuario lo indica
        if accion.lower() == "salir":
            break

        print("Consultando a Ollama...")
        
        # Aquí llamas a la cadena para obtener los resultados
        respuesta = chain.invoke({"accion": str(accion)})  # Suponiendo que respuesta es de tipo Usuario
        user = str(respuesta.usuario)  # Acceder al valor de 'usuario'
        password = respuesta.contrasena  # Acceder al valor de 'contrasena'
        
        # Mostrar los tipos y valores
        print(type(user), user, type(password), password)

    # Verifica si la respuesta tiene los datos correctos
        if user and password is not None:
            # Ejecutar Puppeteer con los datos obtenidos
            resultado = ejecutar_puppeteer(user,password)
        else:
            print("No se pudo obtener el usuario y/o la contraseña. Por favor, proporciona ambos datos.")
        

if __name__ == "__main__":
    main()



Consultando a Ollama...
<class 'str'> ¿Por favor ingresa tu nombre de usuario? <class 'str'> ¿Por favor ingresa tu contraseña?
¿Por favor ingresa tu nombre de usuario? ¿Por favor ingresa tu contraseña?
Inicio de sesión realizado correctamente.
Consultando a Ollama...
<class 'str'> admin <class 'str'> 1234
admin 1234
Inicio de sesión realizado correctamente.
